In [1]:
import dask.array as da
import numba
import numpy as np

# Dask from_delayed

https://docs.dask.org/en/latest/delayed-collections.html

# Dask Block/Chunks

https://docs.dask.org/en/latest/array-stack.html

https://docs.dask.org/en/latest/array-chunks.html

In [2]:
arr0 = da.from_array(np.zeros((3, 4)), chunks=(1, 2))
arr1 = da.from_array(np.ones((3, 4)), chunks=(1, 2))

In [3]:
arr0

dask.array<array, shape=(3, 4), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

In [4]:
data = [

    [arr0, arr1],

    [arr1, arr0]

]

x = da.block(data)

x.shape

(6, 8)

In [5]:
x

dask.array<concatenate, shape=(6, 8), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

# Numba

In [2]:
@numba.jit
def f(x, y):
    # A somewhat trivial example
    return x + y


In [3]:
@numba.jit(numba.int32(numba.int32, numba.int32))
def f(x, y):
    # A somewhat trivial example
    return x + y

In [4]:
import time

x = np.arange(100).reshape(10, 10).astype("float64")

@numba.jit(nopython=True)
def go_fast(a): # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.34642672538757324
Elapsed (after compilation) = 8.225440979003906e-05


In [7]:
%%timeit
t = go_fast(x)

770 ns ± 3.52 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
# Eager compilation
@numba.jit(numba.float64[:, :](numba.float64[:, :]), nopython=True)
def go_fast_2(a): # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace


In [9]:
# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast_2(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast_2(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.0004935264587402344
Elapsed (after compilation) = 0.0001430511474609375


In [10]:
%%timeit
go_fast_2(x)

735 ns ± 1.48 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
